# Simple Column Access Audit

This notebook demonstrates the key linkage between `system.query.history` and `system.access.column_lineage` using the `statement_id` field.

**Important**: Column names may vary at time of execution; run DESCRIBE commands first to see available columns.

In [0]:
from datetime import datetime, timedelta
import pytz

# Configuration
TARGET_CATALOG = "marcin_demo"  # CHANGE THIS
TARGET_SCHEMA = "demo_schema_v2"    # CHANGE THIS

# Time range and current user
END_TIME = datetime.now(pytz.UTC) + timedelta(minutes=5) #fast-forward for example
START_TIME = END_TIME - timedelta(days=1)
CURRENT_USER = spark.sql("SELECT current_user()").collect()[0][0]

print(f"Catalog.Schema: {TARGET_CATALOG}.{TARGET_SCHEMA}")
print(f"User: {CURRENT_USER}")
print(f"Time Range: Last 24 hours")

Catalog.Schema: marcin_demo.demo_schema_v2
User: marcin.jimenez@databricks.com
Time Range: Last 24 hours


In [0]:
%sql
USE CATALOG marcin_demo;
CREATE SCHEMA IF NOT EXISTS demo_schema_v2;
USE marcin_demo.demo_schema_v2;
DROP TABLE IF EXISTS sample_iris_v2;

-- 1) Create the table from the CSV in DBFS
CREATE TABLE IF NOT EXISTS sample_iris_v2 (
  sepal_length DOUBLE,
  sepal_width  DOUBLE,
  petal_length DOUBLE,
  petal_width  DOUBLE,
  species      STRING
)
USING DELTA;

-- -- 2) Tag the sensitive columns
SET TAG ON COLUMN sample_iris_v2.petal_width SENSITIVE;
SET TAG ON COLUMN sample_iris_v2.species SENSITIVE;

In [0]:
 import pandas as pd
 import pyspark.pandas as ps
 from sklearn.datasets import load_iris

 # Load the IRIS dataset
 iris = load_iris()

 # Create a DataFrame
 iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
 iris_df.rename(columns={
     'sepal length (cm)': 'sepal_length',
     'sepal width (cm)': 'sepal_width',
     'petal length (cm)': 'petal_length',
     'petal width (cm)': 'petal_width'
 }, inplace=True)

iris_df['species'] = iris.target.astype(str)

 # Convert to Spark DataFrame and write to Unity Catalog
 ps.from_pandas(iris_df).to_table("marcin_demo.demo_schema_v2.sample_iris_v2", mode="overwrite")

/databricks/python/lib/python3.10/site-packages/pyspark/pandas/utils.py:1026: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/databricks/python/lib/python3.10/site-packages/pyspark/pandas/utils.py:1026: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_table`, the existing index is lost when converting to table.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [0]:
# trigger a search query
display(spark.sql(f"SELECT * FROM {TARGET_CATALOG}.{TARGET_SCHEMA}.sample_iris_v2"))

sepal_length,sepal_width,petal_length,petal_width,species
5.1,3.5,1.4,0.2,0
4.9,3.0,1.4,0.2,0
4.7,3.2,1.3,0.2,0
4.6,3.1,1.5,0.2,0
5.0,3.6,1.4,0.2,0
5.4,3.9,1.7,0.4,0
4.6,3.4,1.4,0.3,0
5.0,3.4,1.5,0.2,0
4.4,2.9,1.4,0.2,0
4.9,3.1,1.5,0.1,0


## 0. View System Tables

In [0]:
# Check what columns exist in column_lineage
print("=== COLUMN LINEAGE TABLE SCHEMA ===")
display(spark.sql("DESCRIBE system.access.column_lineage").limit(50))

=== COLUMN LINEAGE TABLE SCHEMA ===


col_name,data_type,comment
account_id,string,The id of the Databricks account.
metastore_id,string,The id of the Unity Catalog metastore.
workspace_id,string,The id of the workspace
entity_type,string,"The type of entity the lineage transaction was captured from. The supported value is NOTEBOOK, JOB, PIPELINE, DASHBOARD_V3 (AI/BI Dashboard), DBSQL_DASHBOARD (Legacy dashboard), DBSQL_QUERY, OR NULL."
entity_id,string,"The id of the entity the lineage transaction was captured from. If entity_type is NULL, entity_id is NULL."
entity_run_id,string,"id to describe the unique run of the entity, or NULL. This differs for each entity type: Notebook: command_run_id Job: job_run_id Databricks SQL query: statement_id Dashboard: statement_id Legacy dashboard: statement_id Pipeline: pipeline_update_id If entity_type is NULL, entity_run_id is NULL. Records with statement_id and job_run_id can be joined with the query history and jobs system tables respectively."
source_table_full_name,string,Three-level name to identify the source table.
source_table_catalog,string,The catalog of the source table.
source_table_schema,string,The schema of the source table.
source_table_name,string,The name of the source table.


In [0]:
# Check what columns exist in query_history
print("\n=== QUERY HISTORY TABLE SCHEMA ===")
display(spark.sql("DESCRIBE system.query.history").limit(50))


=== QUERY HISTORY TABLE SCHEMA ===


col_name,data_type,comment
account_id,string,ID of the account.
workspace_id,string,The ID of the workspace where the query was run.
statement_id,string,The ID that uniquely identifies the execution of the statement. You can use this ID to find the statement execution in the Query History UI.
executed_by,string,The email address or username of the user who ran the statement.
session_id,string,The Spark session ID.
execution_status,string,The statement termination state. Possible values are: FINISHED: execution was successful FAILED: execution failed with the reason for failure described in the accompanying error message CANCELED: execution was canceled
compute,struct,A struct that represents the type of compute resource used to run the statement and the ID of the resource where applicable. The type value will be WAREHOUSE.
executed_by_user_id,string,The ID of the user who ran the statement.
statement_text,string,"Text of the SQL statement. If you have configured customer-managed keys, statement_text is empty."
statement_type,string,"The statement type. For example: ALTER, COPY, and`INSERT`."


## 1. Sample Column Lineage Data

In [0]:
print("Sample column lineage data:")
display(spark.sql(f"""
SELECT 
  statement_id,
  source_table_full_name,
  source_column_name,
  target_table_full_name,
  target_column_name,
  entity_type
FROM system.access.column_lineage
WHERE source_table_full_name LIKE '{TARGET_CATALOG}.{TARGET_SCHEMA}.%'
LIMIT 5
"""))

Sample column lineage data:


statement_id,source_table_full_name,source_column_name,target_table_full_name,target_column_name,entity_type
30fcb976-477a-4ac4-b8a8-5e4d6855a317,marcin_demo.demo_schema_v2.sample_iris_v2,sepal_length,null,null,NOTEBOOK
30fcb976-477a-4ac4-b8a8-5e4d6855a317,marcin_demo.demo_schema_v2.sample_iris_v2,petal_length,null,null,NOTEBOOK
30fcb976-477a-4ac4-b8a8-5e4d6855a317,marcin_demo.demo_schema_v2.sample_iris_v2,petal_width,null,null,NOTEBOOK
30fcb976-477a-4ac4-b8a8-5e4d6855a317,marcin_demo.demo_schema_v2.sample_iris_v2,sepal_width,null,null,NOTEBOOK
30fcb976-477a-4ac4-b8a8-5e4d6855a317,marcin_demo.demo_schema_v2.sample_iris_v2,species,null,null,NOTEBOOK


## 2. Sample Query History Data

In [0]:
print(f"Sample queries by {CURRENT_USER}:")
display(spark.sql(f"""
SELECT 
  statement_id,
  executed_by,
  start_time,
  workspace_id,
  query_source,
  statement_text
FROM system.query.history
WHERE executed_by = '{CURRENT_USER}'
  AND start_time >= '{START_TIME.isoformat()}'
ORDER BY start_time DESC
LIMIT 50
"""))

Sample queries by marcin.jimenez@databricks.com:


statement_id,executed_by,start_time,workspace_id,query_source,statement_text
7d2fccbb-51ca-4572-8e23-57b5096d95a7,marcin.jimenez@databricks.com,2025-07-23T00:41:36.207Z,984752964297111,"List(List(null, null, null), null, null, null, 3650006914012235, null, null)","display(spark.sql(f"""""" SELECT statement_id, source_table_full_name, source_column_name, target_table_full_name, target_column_name FROM system.access.column_lineage WHERE source_table_full_name LIKE '{TARGET_CATALOG}.{TARGET_SCHEMA}.%' LIMIT 5 """"""))"
555929ad-e540-42c3-86b2-af058c8163ab,marcin.jimenez@databricks.com,2025-07-23T00:40:41.790Z,984752964297111,"List(List(null, null, null), null, null, null, 3650006914012235, null, null)","display(spark.sql(f"""""" SELECT statement_id, executed_by, start_time, workspace_id, query_source, statement_text FROM system.query.history WHERE executed_by = '{CURRENT_USER}' AND start_time >= '{START_TIME.isoformat()}' ORDER BY start_time DESC LIMIT 50 """"""))"
66dd3b59-3ebd-4695-9273-80a4ca2f3dec,marcin.jimenez@databricks.com,2025-07-23T00:40:27.706Z,984752964297111,"List(List(null, null, null), null, null, null, 3650006914012235, null, null)","display(spark.sql(f"""""" SELECT statement_id, executed_by, start_time, workspace_id, query_source, statement_text FROM system.query.history WHERE executed_by = '{CURRENT_USER}' AND start_time >= '{START_TIME.isoformat()}' ORDER BY start_time DESC LIMIT 5 """"""))"
55b01942-8993-4e27-9007-636d5aabfb90,marcin.jimenez@databricks.com,2025-07-23T00:39:53.455Z,984752964297111,"List(List(null, null, null), null, null, null, 3650006914012235, null, null)","display(spark.sql(f"""""" SELECT statement_id, executed_by, start_time, workspace_id, query_source FROM system.query.history WHERE executed_by = '{CURRENT_USER}' AND start_time >= '{START_TIME.isoformat()}' ORDER BY start_time DESC LIMIT 5 """"""))"
249830ea-6ea9-4c64-bc62-8f5d295b92e3,marcin.jimenez@databricks.com,2025-07-23T00:39:31.512Z,984752964297111,"List(List(null, null, null), null, null, null, 3650006914012235, null, null)","spark.sql(f"""""" SELECT statement_id, executed_by, start_time, workspace_id, query_source FROM system.query.history WHERE executed_by = '{CURRENT_USER}' AND start_time >= '{START_TIME.isoformat()}' ORDER BY start_time DESC LIMIT 5 """""").show(truncate=False)"
d914a8e8-72e7-4aab-bf8c-3d0524058b58,marcin.jimenez@databricks.com,2025-07-23T00:38:42.033Z,984752964297111,"List(List(null, null, null), null, null, null, 3650006914012235, null, null)","display(spark.sql(f"""""" SELECT statement_id, source_table_full_name, source_column_name, target_table_full_name, target_column_name FROM system.access.column_lineage WHERE source_table_full_name LIKE '{TARGET_CATALOG}.{TARGET_SCHEMA}.%' LIMIT 5 """"""))"
b72921d9-171a-434c-bbd1-876cd5a05bf4,marcin.jimenez@databricks.com,2025-07-23T00:38:33.083Z,984752964297111,"List(List(null, null, null), null, null, null, 3650006914012235, null, null)","display(spark.sql(f""SELECT * FROM {TARGET_CATALOG}.{TARGET_SCHEMA}.sample_iris_v2""))"
6370f434-db6f-436e-a3f0-44eedb20ff43,marcin.jimenez@databricks.com,2025-07-23T00:38:20.646Z,984752964297111,"List(List(null, null, null), null, null, null, 3650006914012235, null, null)","display(spark.sql(f""SELECT * FROM {TARGET_CATALOG}.{TARGET_SCHEMA}.sample_iris_v2""))"
d70908ad-c8b9-4b65-b26e-75b64c5a9494,marcin.jimenez@databricks.com,2025-07-23T00:36:26.941Z,984752964297111,"List(List(null, null, null), null, null, null, 3650006914012235, null, null)","CURRENT_USER = spark.sql(""SELECT current_user()"").collect()[0][0]"
577ae8e6-0095-400e-ab3c-4228eda1f498,marcin.jimenez@databricks.com,2025-07-23T00:36:05.892Z,984752964297111,"List(List(null, null, null), null, null, null, 3650006914012235, null, null)","display(spark.sql(f"""""" SELECT statement_id, source_table_full_name, source_column_name, target_table_full_name, target_column_name FROM system.access.column_lineage WHERE source_table_full_name LIKE '{TARGET_CATALOG}.{TARGET_SCHEMA}.%' 

## 3. Simple Join - Show Column Access

In [0]:
# Join the tables on statement_id
print(f"Columns accessed by {CURRENT_USER}:")
display(spark.sql(f"""
SELECT 
  qh.start_time,
  qh.executed_by,
  cl.source_table_full_name,
  cl.source_column_name,
  cl.target_table_full_name,
  cl.target_column_name
FROM system.query.history qh
INNER JOIN system.access.column_lineage cl
  ON qh.statement_id = cl.statement_id
WHERE qh.executed_by = '{CURRENT_USER}'
  AND qh.start_time >= '{START_TIME.isoformat()}'
  AND cl.source_table_full_name LIKE '{TARGET_CATALOG}.{TARGET_SCHEMA}.%'
ORDER BY qh.start_time DESC
LIMIT 20
"""))

Columns accessed by marcin.jimenez@databricks.com:


start_time,executed_by,source_table_full_name,source_column_name,target_table_full_name,target_column_name
2025-07-23T00:34:13.658Z,marcin.jimenez@databricks.com,marcin_demo.demo_schema_v2.sample_iris_v2,petal_width,null,null
2025-07-23T00:34:13.658Z,marcin.jimenez@databricks.com,marcin_demo.demo_schema_v2.sample_iris_v2,sepal_width,null,null
2025-07-23T00:34:13.658Z,marcin.jimenez@databricks.com,marcin_demo.demo_schema_v2.sample_iris_v2,petal_length,null,null
2025-07-23T00:34:13.658Z,marcin.jimenez@databricks.com,marcin_demo.demo_schema_v2.sample_iris_v2,species,null,null
2025-07-23T00:34:13.658Z,marcin.jimenez@databricks.com,marcin_demo.demo_schema_v2.sample_iris_v2,sepal_length,null,null


## 4. Find Sensitive Columns

In [0]:
# List columns tagged as sensitive
print("Sensitive columns in our schema:")
sensitive_df = spark.sql(f"""
SELECT 
  CONCAT(catalog_name, '.', schema_name, '.', table_name) as table_name,
  column_name,
  tag_name,
  tag_value
FROM system.information_schema.column_tags
WHERE catalog_name = '{TARGET_CATALOG}'
  AND schema_name = '{TARGET_SCHEMA}'
  AND tag_name IN ('PHI', 'PII', 'SENSITIVE', 'CONFIDENTIAL')
ORDER BY table_name, column_name
""")

display(sensitive_df)
print(f"\nTotal sensitive columns: {sensitive_df.count()}")

Sensitive columns in our schema:


table_name,column_name,tag_name,tag_value
marcin_demo.demo_schema_v2.sample_iris_v2,petal_width,SENSITIVE,
marcin_demo.demo_schema_v2.sample_iris_v2,species,SENSITIVE,



Total sensitive columns: 2


## 5. Main Query - Who Accessed Sensitive Columns?

In [0]:
# Putting it all together
result = spark.sql(f"""
WITH sensitive_cols AS (
  SELECT 
    CONCAT(catalog_name, '.', schema_name, '.', table_name) as table_name,
    column_name,
    tag_name
  FROM system.information_schema.column_tags
  WHERE catalog_name = '{TARGET_CATALOG}'
    AND schema_name = '{TARGET_SCHEMA}'
    AND tag_name IN ('PHI', 'PII', 'SENSITIVE')
)
SELECT 
  qh.start_time as access_time,
  qh.executed_by as user_name,
  cl.source_table_full_name as table_name,
  cl.source_column_name as column_name,
  cl.entity_type as access_type,
  sc.tag_name
FROM system.query.history qh
INNER JOIN system.access.column_lineage cl
  ON qh.statement_id = cl.statement_id
INNER JOIN sensitive_cols sc
  ON cl.source_table_full_name = sc.table_name
  AND cl.source_column_name = sc.column_name
WHERE qh.start_time >= '{START_TIME.isoformat()}'
ORDER BY qh.start_time DESC
LIMIT 100
""")
display(result)

access_time,user_name,table_name,column_name,access_type,tag_name
2025-07-23T00:38:33.083Z,marcin.jimenez@databricks.com,marcin_demo.demo_schema_v2.sample_iris_v2,petal_width,NOTEBOOK,SENSITIVE
2025-07-23T00:38:33.083Z,marcin.jimenez@databricks.com,marcin_demo.demo_schema_v2.sample_iris_v2,species,NOTEBOOK,SENSITIVE
2025-07-23T00:38:20.646Z,marcin.jimenez@databricks.com,marcin_demo.demo_schema_v2.sample_iris_v2,species,NOTEBOOK,SENSITIVE
2025-07-23T00:38:20.646Z,marcin.jimenez@databricks.com,marcin_demo.demo_schema_v2.sample_iris_v2,petal_width,NOTEBOOK,SENSITIVE
2025-07-23T00:34:13.658Z,marcin.jimenez@databricks.com,marcin_demo.demo_schema_v2.sample_iris_v2,petal_width,NOTEBOOK,SENSITIVE
2025-07-23T00:34:13.658Z,marcin.jimenez@databricks.com,marcin_demo.demo_schema_v2.sample_iris_v2,species,NOTEBOOK,SENSITIVE


In [0]:

from pyspark.sql.functions import count, countDistinct

if result.count() > 0:    
    print("\n📊 SUMMARY BY USER:")
    display(result.groupBy("user_name").agg(
        count("*").alias("access_count"),
        countDistinct("column_name").alias("unique_columns")
    ).orderBy("access_count", ascending=False))
else:
    print("✅ No sensitive column access detected in the specified time period.")


📊 SUMMARY BY USER:


user_name,access_count,unique_columns
marcin.jimenez@databricks.com,6,2
